## Neighbourhood Analysis in HK

In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle json files

import re #library for regular expression operations to search strings

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

SyntaxError: invalid syntax (<ipython-input-5-5cf841375e65>, line 13)

In [4]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


ModuleNotFoundError: No module named 'geopy'

In [6]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle json files

import re #library for regular expression operations to search strings

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab
usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


In [9]:
areas = ['HK','Kowloon','NT']

link = "https://www.elections.gov.hk/dc2019/eng/poll_{}.html"
links = dict(zip(areas,[[],[],[]]))

#HK for Hong Kong island, Kowloon for Kowloon, NT for New Territorries
#HK, Kowloon and NT are three main areas in Hong Kong
codes = {'HK':['A','B','C','D'],
        'Kowloon':['E','F','G','H','J'],
        'NT':['K','L','M','N','P','Q','R','S','T']}

for area in areas:
    for i in codes[area]:
        links[area].append(link.format(i))

print(links['HK'],links['Kowloon'],links['NT'])

['https://www.elections.gov.hk/dc2019/eng/poll_A.html', 'https://www.elections.gov.hk/dc2019/eng/poll_B.html', 'https://www.elections.gov.hk/dc2019/eng/poll_C.html', 'https://www.elections.gov.hk/dc2019/eng/poll_D.html'] ['https://www.elections.gov.hk/dc2019/eng/poll_E.html', 'https://www.elections.gov.hk/dc2019/eng/poll_F.html', 'https://www.elections.gov.hk/dc2019/eng/poll_G.html', 'https://www.elections.gov.hk/dc2019/eng/poll_H.html', 'https://www.elections.gov.hk/dc2019/eng/poll_J.html'] ['https://www.elections.gov.hk/dc2019/eng/poll_K.html', 'https://www.elections.gov.hk/dc2019/eng/poll_L.html', 'https://www.elections.gov.hk/dc2019/eng/poll_M.html', 'https://www.elections.gov.hk/dc2019/eng/poll_N.html', 'https://www.elections.gov.hk/dc2019/eng/poll_P.html', 'https://www.elections.gov.hk/dc2019/eng/poll_Q.html', 'https://www.elections.gov.hk/dc2019/eng/poll_R.html', 'https://www.elections.gov.hk/dc2019/eng/poll_S.html', 'https://www.elections.gov.hk/dc2019/eng/poll_T.html']


In [10]:
header = {
    'User-Agent': 'Mozilla/5.0'
}

page = requests.post(link, headers=header)
soup = BeautifulSoup(page.content)


tables = {'HK':dict(zip(codes['HK'],[ [] for _ in range(len(codes['HK']))])),
          'Kowloon':dict(zip(codes['Kowloon'],[ [] for _ in range(len(codes['Kowloon']))])),
          'NT':dict(zip(codes['NT'],[ [] for _ in range(len(codes['NT']))]))}


for area in tables.keys():
    for key in tables[area].keys():
        page = requests.post(link.format(key), headers=header)
        soup = BeautifulSoup(page.content)
        tables[area][key] = soup.find(id="table-district-member")
        tables[area][key] = soup.findAll('td')

In [11]:
district = {'A':'Central & Western District',
            'B':'Wan Chai District',
            'C':'Eastern District',
            'D':'Southern District',
            'E':'Yau Tsim Mong District',
            'F':'Sham Shui Po District',
            'G':'Kowloon City District',
            'H':'Wong Tai Sin District',
            'J':'Kwun Tong District',
            'K':'Tsuen Wan District',
            'L':'Tuen Mun District',
            'M':'Yuen Long District',
            'N':'North District',
            'P':'Tai Po District',
            'Q':'Sai Kung District',
            'R':'Sha Tin District',
            'S':'Kwai Tsing District',
            'T':'Islands District'}

In [12]:
columns = ["District","Constituency","Code","Polling Station","Address"]
dfs = {'HK':pd.DataFrame(columns = columns),
      'Kowloon':pd.DataFrame(columns = columns),
      'NT':pd.DataFrame(columns = columns)}


entry = [None]*5 #temporarily storing the row entry
for area in areas:
    
    row_index = 0
    
    for key in tables[area].keys():
        
        constituency_index = 0 #j is the constituency index for each district
        #print(key)
        
        for i in range(len(tables[area][key])): #i is the number of strings of each district
            
            #print(tables[area][key][i].text)
            if tables[area][key][i].text == key + "%02d" % int(constituency_index+1):
                flag = tables[area][key][i].text
                constituency_index = constituency_index+1
                column = 1
                Polling_station_index = 1 #l is the number of polling station with the same constituency
                #print(tables[i].text)

            elif tables[area][key][i].text[0:5] == flag + "%02d" % Polling_station_index:
                column = 2
                entry[column] = tables[area][key][i].text
                
                column = column + 1
                
            
            elif tables[area][key][i].text == 'Map':
                #print(entry[2])
                #print(tables[area][key][i].text)
                dfs[area].loc[row_index]=[district[key],entry[1],entry[2],entry[3],entry[4]]
                Polling_station_index = Polling_station_index + 1
                row_index = row_index + 1
                #print(row_index)
        
            else:
                entry[column] = tables[area][key][i].text
                column = column + 1
        
#check the data entries on HK island
dfs['HK']

,District,Constituency,Code,Polling Station,Address
0,Central & Western District,CHUNG WAN,A0101,H6 Conet,"G/F, The Center, 99 Queen's Road Central, Cent..."
1,Central & Western District,CHUNG WAN,A0102,Hong Kong Visual Arts Centre,"4/F, 7A Kennedy Road, Central, Hong Kong"
2,Central & Western District,MID LEVELS EAST,A0201,Raimondi College,"2 Robinson Road, Hong Kong"
3,Central & Western District,CASTLE ROAD,A0301,Hong Kong True Light Kindergarten (Caine Road),"G/F - 2/F, 75 Caine Road, Hong Kong"
4,Central & Western District,CASTLE ROAD,A0302,Ying Wa Girls' School,"76 Robinson Road, Mid-Levels, Hong Kong"
5,Central & Western District,PEAK,A0401,Hong Kong Park Sports Centre,"29 Cotton Tree Drive, Central, Hong Kong"
6,Central & Western District,PEAK,A0402,German Swiss International School Peak Campus,"Upper Building, German Swiss International Sch..."
7,Central & Western District,UNIVERSITY,A0501,Sai Ying Pun Community Complex Community Hall,"3/F, Sai Ying Pun Community Complex, 2 High St..."
8,Central & Western District,KWUN LUNG,A0601,Smithfield Sports Centre,"4/F, Smithfield Municipal Services Building, 1..."
9,Central & Western District,KENNEDY TOWN & MOUNT DAVIS,A0701,HKYWCA Western District Integrated Social Serv...,"Flat A, 1/F, Block 1, Centenary Mansion, 9-15 ..."


In [13]:
find_num = re.compile(r'(.*?)[\d]', re.S)

for area in areas:
    
    #to remove floor number and building names
    #Note this step does not work for entries that do not have street numbers.
    #These entries will be revised again by following steps.
    k=0
    while k < len(dfs[area]['Address']):
        k = 0
        for i, address in enumerate(dfs[area]['Address']):
            try:
                
                address = address.replace(address[0:int(re.search(r', \d',address).start())+2],"")
                dfs[area]['Address'][i] = address
            
            except: 
                k = k + 1

    #Simplify the building number 
    k = 0
    while k < len(dfs[area]['Address']):
        k = 0
        for i, address in enumerate(dfs[area]['Address']):
            try:
                
                address = address.replace(address[0:int(re.search(r'- \d',address).start()+1)],"")
                dfs[area]['Address'][i] = address
            
            except: 
                k = k + 1
    
    #Remove the floor information again
    k = 0
    while k < len(dfs[area]['Address']):
        k = 0
        for i, address in enumerate(dfs[area]['Address']):
            try:

                address = address.replace(address[0:int(re.search(r'/F,',address).start()+4)],"")
                dfs[area]['Address'][i] = address
            
            except: 
                k = k + 1            
            
dfs['HK']

,District,Constituency,Code,Polling Station,Address
0,Central & Western District,CHUNG WAN,A0101,H6 Conet,"99 Queen's Road Central, Central, Hong Kong"
1,Central & Western District,CHUNG WAN,A0102,Hong Kong Visual Arts Centre,"7A Kennedy Road, Central, Hong Kong"
2,Central & Western District,MID LEVELS EAST,A0201,Raimondi College,"2 Robinson Road, Hong Kong"
3,Central & Western District,CASTLE ROAD,A0301,Hong Kong True Light Kindergarten (Caine Road),"75 Caine Road, Hong Kong"
4,Central & Western District,CASTLE ROAD,A0302,Ying Wa Girls' School,"76 Robinson Road, Mid-Levels, Hong Kong"
5,Central & Western District,PEAK,A0401,Hong Kong Park Sports Centre,"29 Cotton Tree Drive, Central, Hong Kong"
6,Central & Western District,PEAK,A0402,German Swiss International School Peak Campus,"11 Guildford Road, The Peak, Hong Kong"
7,Central & Western District,UNIVERSITY,A0501,Sai Ying Pun Community Complex Community Hall,"2 High Street, Sai Ying Pun, Hong Kong"
8,Central & Western District,KWUN LUNG,A0601,Smithfield Sports Centre,"12K Smithfield, Kennedy Town, Hong Kong"
9,Central & Western District,KENNEDY TOWN & MOUNT DAVIS,A0701,HKYWCA Western District Integrated Social Serv...,"9-15 Victoria Road, Western District, Hong Kong"


In [14]:
latitude = []
longitude = []
df_location = {'HK':pd.DataFrame(columns = ['Address','Latitude','Longitude']),
               'Kowloon':pd.DataFrame(columns = ['Address','Latitude','Longitude']),
               'NT':pd.DataFrame(columns = ['Address','Latitude','Longitude'])}

for area in areas:
    for i, address in enumerate(dfs[area]['Address']):
        try:
            geolocator = Nominatim(user_agent="hk_explorer")
            location = geolocator.geocode(address)
            df_location[area].loc[i]=[address,location.latitude,location.longitude]
        except:
            #Print the addresses that geopy cannot get the locations for
            print(area,i,address)
    
df_location['HK']

HK 24 Moreton Terrace, Underneath Tai Hang Road, Flyover, Causeway Bay, Hong Kong (Beside Hong Kong Central Library)
HK 26 62 Tin Hau Temple Road, Tin Hau, Hong Kong
HK 29 17 Tai Hang Drive, Causeway Bay, Hong Kong
HK 36 5 Tai Yue Avenue, Taikoo Shing, Hong Kong
HK 38 2 Tai Wing Avenue, Taikoo Shing, Quarry Bay, Hong Kong
HK 46 6 Shing Hong Lane, Heng Fa Chuen, Hong Kong
HK 47 Tsui Wan Estate,Tsui Wan Street, Chai Wan, Hong Kong
HK 48 9 Harmony Road, Siu Sai Wan, Chai Wan, Hong Kong
HK 49 15 Siu Sai Wan Road, Chai Wan, Hong Kong
HK 50 31 Siu Sai Wan Road, Chai Wan, Hong Kong
HK 53 38 Quarry Bay Street, Hong Kong
HK 60 82 Marble Road, North Point, Hong Kong
HK 66 Hing Cho House, Hing Tung Estate, Shau Kei Wan, Hong Kong
HK 67 King Hei House, Tung Hei Court, Shau Kei Wan, Hong Kong
HK 70 Wing B, Yiu Lok House, Yiu Tung Estate, Shau Kei Wan, Hong Kong
HK 71 Level 3, Community Building, Hing Man Estate, Chai Wan, Hong Kong
HK 72 Cheuk Wah House, Hing Wah(I) Estate, Chai Wan, Hong Kong
HK 7

,Address,Latitude,Longitude
0,"99 Queen's Road Central, Central, Hong Kong",22.283650,114.154940
1,"7A Kennedy Road, Central, Hong Kong",22.276706,114.159361
2,"2 Robinson Road, Hong Kong",22.278408,114.153679
3,"75 Caine Road, Hong Kong",22.281955,114.151291
4,"76 Robinson Road, Mid-Levels, Hong Kong",22.282718,114.147223
5,"29 Cotton Tree Drive, Central, Hong Kong",22.277250,114.159306
6,"11 Guildford Road, The Peak, Hong Kong",22.265953,114.161964
7,"2 High Street, Sai Ying Pun, Hong Kong",22.284792,114.144139
8,"12K Smithfield, Kennedy Town, Hong Kong",22.281721,114.128484
9,"9-15 Victoria Road, Western District, Hong Kong",22.281902,114.125827


In [15]:
#Integrate the location information into the dataframe
for area in areas:
    dfs[area] = dfs[area].join(df_location[area].set_index('Address'), on='Address')

#Drop the entries without location information
for area in areas: 
    dfs[area].dropna(inplace=True)
    print(area,dfs[area].shape)

HK (77, 7)
Kowloon (98, 7)
NT (212, 7)


In [16]:
#Get the location of Hong Kong (The location of Kowloon is used)
address = 'Kowloon, Hong Kong'

geolocator = Nominatim(user_agent="hk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hong Kong island are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hong Kong island are 22.3116452, 114.1767885.


In [17]:
# create map of Toronto City using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
colors = ['red','blue','green']
for area, color in zip(areas, colors):
    for lat, lng, constituency, address in zip(dfs[area]['Latitude'], dfs[area]['Longitude'], dfs[area]['Constituency'], dfs[area]['Address']):
        label = '{}, {}'.format(constituency, address)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map)  

    
map

In [19]:
CLIENT_ID = '4ORWWNFRNAWZKEIWSUVLBVB4FJDXYDWYGEEYHX1PAQ1LY5NE' # your Foursquare ID
CLIENT_SECRET = 'ODJLX4FEQLR51DL55QU0RNUIU2YEJIH1EIDTA4RACRRXYW53' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4ORWWNFRNAWZKEIWSUVLBVB4FJDXYDWYGEEYHX1PAQ1LY5NE
CLIENT_SECRET:ODJLX4FEQLR51DL55QU0RNUIU2YEJIH1EIDTA4RACRRXYW53


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
venues=dict(zip(areas,[[],[],[]]))

for area in areas:
    venues[area] = getNearbyVenues(names=dfs[area]['Polling Station'],
                                       latitudes=dfs[area]['Latitude'],
                                       longitudes=dfs[area]['Longitude']
                                      )

ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=4ORWWNFRNAWZKEIWSUVLBVB4FJDXYDWYGEEYHX1PAQ1LY5NE&client_secret=ODJLX4FEQLR51DL55QU0RNUIU2YEJIH1EIDTA4RACRRXYW53&v=20180605&ll=22.31402395,114.22708903928974&radius=500&limit=100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001D33CB2C1C8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [26]:
venues['NT']

[]

In [27]:
for area in areas:
    venues[area].groupby('Neighborhood').count()

#For following analysis, venues_tot is used.
venues_tot=pd.concat([venues['HK'],venues['Kowloon'],venues['NT']])

AttributeError: 'list' object has no attribute 'groupby'

In [28]:
merged_tot = pd.concat([merged['HK'],merged['Kowloon'],merged['NT']])
x,y=[],[]
for i in range(5):
    temp = merged_tot[merged_tot['Cluster Labels']==i].groupby('1st Most Common Venue').count().drop(index=['Fast Food Restaurant','Chinese Restaurant'])
    common_venues = pd.DataFrame(columns = ['Frequence'])
    common_venues['Frequence'] = temp['Code']
    x.append(common_venues.sort_values('Frequence',ascending=False)[0:5].index)
    y.append(common_venues.sort_values('Frequence',ascending=False)[0:5]['Frequence'])
    
plt.barh(x[4],y[4],label='Cluster 4')
plt.tick_params(labelsize=16)
plt.legend(prop={'size':16})

NameError: name 'merged' is not defined